In [55]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from webdriver_manager.chrome import ChromeDriverManager

import dataclasses
import csv
import time

from bs4 import BeautifulSoup, Tag

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Downloading: 100%|██████████| 6.81M/6.81M [00:00<00:00, 48.4MB/s]
C:\Users\Braed\AppData\Local\Temp\ipykernel_21412\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [41]:
yahoo_finance_news_url: str = "https://finance.yahoo.com/news"

In [39]:
@dataclasses.dataclass
class yahoo_article_info:
    title: str
    link: str
    source: str
    age_at_pull: str
    category: str
    description: str



In [48]:
def get_article_data(src: Tag):# -> yahoo_article_info:
    header = src.find("h3")
    title = header.find("a").text
    href = header.find("a")["href"]
    link = yahoo_finance_news_url + href
    spans = src.find_all("span")
    source = spans[0].text
    age_at_pull = spans[1].text
    category = src.find("div", {"data-test-locator":"catlabel"}).text
    description = src.find("p").text

    result = yahoo_article_info(title=title, link=link, source=source, age_at_pull=age_at_pull, category=category, description=description)
    return result

In [5]:
driver.get(yahoo_finance_news_url)

for _ in range(10):
    body = driver.find_element(By.TAG_NAME, "body")
    body.send_keys(Keys.END)
    time.sleep(2)

In [6]:
html: str = driver.page_source

In [7]:
soup = BeautifulSoup(html, 'html.parser')

In [8]:
main_soup = soup.find("div", {"id":"Main"})

In [11]:
articles: list = []
if main_soup and type(main_soup) == Tag:
    articles = main_soup.find_all("li", {"class":"js-stream-content"})

In [52]:
print(get_article_data(articles[0]))

article_info_list: list[yahoo_article_info] = []
i: int = 0
while len(article_info_list) < 100 or i >= len(articles):
    try:
        article_info_list.append(get_article_data(articles[i]))
    except:
        pass
    i += 1

yahoo_article_info(title="UPDATE 1-Brazil's new fiscal rules stricter than they look, finance ministry official says", link='https://finance.yahoo.com/news/news/1-brazils-fiscal-rules-stricter-151048046.html', source='Reuters', age_at_pull='9 minutes ago', category='World', description="Brazil's newly introduced fiscal framework is stricter than they appear and will require a discussion of important spending cuts, Gabriel Galipolo, the executive secretary of the Finance Ministry, said on Tuesday.  Lawmakers in the lower house of Congress last week passed the main text of legislation that is set to replace the current spending cap, which has been breached several times in recent years to allow higher government spending.")


In [57]:
with open('yahoo_finance_articles.csv', 'w', newline='') as f:
    fields = [field.name for field in dataclasses.fields(yahoo_article_info)]
    w = csv.DictWriter(f, fields)
    w.writeheader()
    w.writerows([dataclasses.asdict(info) for info in article_info_list])